# Example: simple fit using Lorentzian with Flat background model and lmfit

<a id='Table of Contents'></a><h1>Table of Contents</h1>
- <a href='#introduction'>Introduction</a>
- <a href='#imports'>Importing the required libraries</a>
- <a href='#anim_plot'>Plot of the fitting model</a>
- <a href='#ref_data'>Creating the reference data</a>
- <a href='#fitting'>Setting and fitting </a>  
- <a href='#plotting_results'>Plotting the results</a>  

(<a href='#Table of Contents'>Top</a>)<a id='introduction'></a><h2>Introduction</h2>
<div class="alert alert-block alert-info">
The objective of this notebook is to show how the models of 
the [QENSlibrary](https://github.com/QENSlibrary/QENSmodels) and how t ocmbine them. Here, we use the `Lorentzian` profile and a flat background (created from `background_polynomials`), to perform some fits.


[`lmfit`](https://lmfit.github.io/lmfit-py/) was used for fitting.
</div>

(<a href='#Table of Contents'>Top</a>)<a id='imports'></a><h2>Importing the required librairies</h2>

In [ ]:
# import python modules for plotting, fitting
from __future__ import print_function
import numpy as np
%matplotlib inline
import matplotlib.pyplot as plt

# for interactive plot
import ipywidgets

# import model from QENS library
import QENSmodels

In [ ]:
# install lmfit (if not already installed)
import sys
import pkgutil

if not pkgutil.find_loader("lmfit"):
    !{sys.executable} -m pip install lmfit

In [ ]:
# required imports from lmfit
from lmfit import Model, Parameters

(<a href='#Table of Contents'>Top</a>)<a id='anim_plot'></a><h2>Plot of the fitting model</h2>

The widget below shows the lorentzian peak shape function imported from QENSmodels where the function's parameters *Scale*, *Center* and *FWHM* can be varied.

In [ ]:
xx = np.linspace(-10,10,100)

def interactive_plot(scale, center, hwhm, backgd):
    plt.plot(xx, 
             QENSmodels.lorentzian(xx, float(scale), float(center), float(hwhm))
            + QENSmodels.background_polynomials(xx,float(backgd))
            )
    plt.ylabel('lorentzian(x,{scale},{center},{hwhm})+{backgd}'.
               format(scale=scale, center=center, hwhm=hwhm, backgd=backgd))
    plt.xlabel('x')
    plt.grid()
    plt.show()

interactive_plot = ipywidgets.interactive(interactive_plot, 
                                          scale=(1, 10), 
                                          center=(0, 10), 
                                          hwhm=(1, 5),
                                         backgd=(-1,1))

def reset_values(b):
    """Reset the interactive plots to inital values."""
    interactive_plot.children[0].value = 5
    interactive_plot.children[1].value = 5
    interactive_plot.children[2].value = 3
    interactive_plot.children[2].value = 0

reset_button = ipywidgets.Button(description = "Reset")
reset_button.on_click(reset_values)

output = interactive_plot.children[-1]
output.layout.height = '270px'
display(interactive_plot, reset_button)

(<a href='#Table of Contents'>Top</a>)<a id='ref_data'></a><h2>Creating the reference data</h2>

In [ ]:
# Create array of reference data: noisy lorentzian with background
xx = np.linspace(-5,5,100)
lorentzian_noisy = QENSmodels.lorentzian(xx, 0.89, -0.025, 0.45)*(1+0.1*np.random.normal(0,1,100)) + 0.5+ 0.01*np.random.normal(0,1,100)

(<a href='#Table of Contents'>Top</a>)<a id='fitting'></a><h2>Setting and fitting</h2>

In [ ]:
def flat_background(x, A0):
    """ Define flat background ot be added to fitting model"""
    return QENSmodels.background_polynomials(x, A0)

In [ ]:
gmodel = Model(QENSmodels.lorentzian) + Model(flat_background)
print('Names of parameters:', gmodel.param_names)
print('Independent variable(s):', gmodel.independent_vars)

initial_parameters_values = [1, 0.2, 0.5, 0.33]

# Fit
result = gmodel.fit(lorentzian_noisy, x=xx, 
                    scale=initial_parameters_values[0], 
                    center=initial_parameters_values[1],
                    hwhm=initial_parameters_values[2],
                    A0=initial_parameters_values[3])

# Plots

# to increase vertical spacing between plots
plt.subplots_adjust(left=None, bottom=None, right=None, top=None,
                wspace=None, hspace=0.75)

# Initial model and reference data
plt.subplot(2, 1, 1)
plt.plot(xx, lorentzian_noisy, 'b-', label='reference data')
plt.plot(xx, result.init_fit, 'k--', label='model with initial guesses')
plt.xlabel('x')
plt.grid()
plt.title('Initial model and reference data')
plt.legend(bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0.)

# Fitting result and reference data
plt.subplot(2, 1, 2)
plt.plot(xx, lorentzian_noisy, 'b-', label='reference data')
plt.plot(xx, result.best_fit, 'r.', label='fitting result')
plt.title('Fit result and reference data')
plt.legend(bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0.)
plt.xlabel('x')
plt.grid()

plt.show()

(<a href='#Table of Contents'>Top</a>)<a id='plotting_results'></a><h2>Plotting results </h2>
using methods implemented in `lmfit`

In [ ]:
# display result
print('Result of fit:\n',result.fit_report())

# plot
result.plot()